In [3]:
#Import Libraries and print their versions
%run imports.py

Python version:     3.12.7
pymc version:       5.23.0
numpy version:      1.26.4
pandas version:     2.2.2
arviz version:      0.21.0
matplotlib version: 3.9.2


In [23]:
%run model_B02.py

In [24]:
def calculate_odds_ratio(idata, var_name="effect"):
    """
    Calculates the odds ratio and its 95% HDI from the posterior samples.

    Args:
        idata (az.InferenceData): InferenceData object.
        var_name (str, optional): Name of the parameter. Defaults to "effect".

    Returns:
        pd.DataFrame: DataFrame with mean, lower, and upper HDI for the odds ratio.
                      Returns an empty DataFrame if there's an error.
    """

    try:
        posterior_samples = idata.posterior[var_name].values.flatten()
        odds_ratios = np.exp(posterior_samples)
        hdi = az.hdi(odds_ratios, hdi_prob=0.95)
        mean_or = np.mean(odds_ratios)

        or_summary = pd.DataFrame({
            "mean": [mean_or],
            "hdi_lower": [hdi[0]],
            "hdi_upper": [hdi[1]]
        })
        return or_summary

    except KeyError:
        print(f"Variable '{var_name}' not found in posterior.")
        return pd.DataFrame()  # Return empty DataFrame to signal error


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import arviz as az

def run_and_summarize(data, model_func, label="b02", output_dir="../results", **model_kwargs):
    """
    Runs the specified PyMC model function with data and optional model parameters,
    summarizes results, and saves output files including plots, summaries, and metrics.

    Parameters:
        data (dict): Dictionary with keys 'predictor' and 'outcome'.
        model_func (function): PyMC model function to use.
        label (str): Label to tag output files.
        output_dir (str): Directory to save outputs.
        **model_kwargs: Additional keyword arguments passed to model_func.

    Returns:
        idata (InferenceData): ArviZ inference data object.
        waic (ELPDData): WAIC estimate.
        loo (ELPDData): LOO estimate.
        bic (float): Bayesian Information Criterion.
    """
    os.makedirs(output_dir, exist_ok=True)

    #Call the model function with additional model parameters
    model, idata, waic, loo, bic = model_func(
        data["predictor"],
        data["outcome"],
        **model_kwargs  # Pass arguments like variant="B01", seed=123, etc.
    )

    # Save InferenceData
    idata_path = os.path.join(output_dir, f"idata_{label}.nc")
    idata.to_netcdf(idata_path)

    # Determine var_names based on model
    var_names = ["intercept", "level_effects"]
    if "effect" in idata.posterior:
        var_names += ["effect", "sd_fluctuation"]
    if "p" in idata.posterior:
        var_names.append("p")

    # Remove problematic or degenerate variables
    clean_var_names = []
    for var in var_names:
        if var in idata.posterior:
            values = idata.posterior[var].values
            if np.isnan(values).all():
                print(f"⚠️ Skipping '{var}': all NaNs in posterior.")
            elif np.allclose(values, values.mean()):
                print(f"⚠️ Skipping '{var}': constant values in posterior.")
            else:
                clean_var_names.append(var)
        else:
            print(f"⚠️ Skipping '{var}': not found in posterior.")

    # Save Summary
    summary = az.summary(idata, var_names=clean_var_names)
    summary_path = os.path.join(output_dir, f"survey_summary_{label}.csv")
    summary.to_csv(summary_path)

    # Calculate and save Odds Ratio summary (only for B02)
    if "effect" in idata.posterior:
        try:
            or_summary = calculate_odds_ratio(idata)
            if not or_summary.empty:
                or_summary_path = os.path.join(output_dir, f"survey_or_summary_{label}.csv")
                or_summary.to_csv(or_summary_path)
            else:
                print("Odds ratio calculation failed. Not saving.")
        except Exception as e:
            print(f"❌ Error calculating odds ratio: {e}")
    else:
        print("Model does not have 'effect' parameter. Skipping OR calculation.")

    # Save WAIC, LOO, BIC as a markdown file
    metrics_path = os.path.join(output_dir, f"survey_model_metrics_{label}.md")
    with open(metrics_path, "w") as f:
        f.write(f"## Model {label.upper()} Metrics\n\n")
        f.write("### WAIC\n```\n" + waic.to_string() + "\n```\n")
        f.write("### LOO\n```\n" + loo.to_string() + "\n```\n")
        f.write(f"### BIC\n```\n\nBIC = {bic:.2f}\n```")

    # Trace plot (excluding 'p')
    trace_vars = [v for v in clean_var_names if v != "p"]
    if trace_vars:
        trace_plot_path = os.path.join(output_dir, f"survey_trace_{label}.png")
        az.plot_trace(idata, var_names=trace_vars)
        plt.tight_layout()
        plt.savefig(trace_plot_path, dpi=300)
        plt.close()

    # Posterior plot for effect (if exists)
    if "effect" in idata.posterior:
        effect_plot_path = os.path.join(output_dir, f"survey_posterior_effect_{label}.png")
        az.plot_posterior(idata, var_names=["effect"])
        plt.tight_layout()
        plt.savefig(effect_plot_path, dpi=300)
        plt.close()

    # Posterior plot for a subset of 'p'
    if "p" in idata.posterior:
        try:
            obs_count = idata.posterior["p"].shape[-1]
            plot_indices = np.random.choice(obs_count, size=min(20, obs_count), replace=False)
            posterior_plot_path = os.path.join(output_dir, f"survey_posterior_p_{label}.png")
            az.plot_posterior(idata, var_names=["p"], coords={"obs": plot_indices})
            plt.tight_layout()
            plt.savefig(posterior_plot_path, dpi=300)
            plt.close()
        except Exception as e:
            print(f"⚠️ Failed to plot 'p': {e}")

    # Posterior predictive check
    try:
        ppc_plot_path = os.path.join(output_dir, f"survey_ppc_{label}.png")
        az.plot_ppc(idata)
        plt.tight_layout()
        plt.savefig(ppc_plot_path, dpi=300)
        plt.close()
    except Exception as e:
        print(f"⚠️ Failed to plot PPC: {e}")

    # Rank plot for diagnostics
    try:
        rank_plot_path = os.path.join(output_dir, f"survey_rank_{label}.png")
        az.plot_rank(idata, var_names=trace_vars)
        plt.tight_layout()
        plt.savefig(rank_plot_path, dpi=300)
        plt.close()
    except Exception as e:
        print(f"⚠️ Failed to create rank plot: {e}")

    print(f"✅ Saved all outputs for model {label.upper()} in '{output_dir}' folder.")
    return idata, waic, loo, bic

In [ ]:
import arviz as az
import numpy as np
import matplotlib.pyplot as plt

def run_model_diagnostics(idata, model_name="Model", output_dir="../results"):
    """
    Performs standard Bayesian diagnostics on a PyMC InferenceData object.
    Saves plots and prints summary messages for convergence, divergences, etc.
    """
    import os
    os.makedirs(output_dir, exist_ok=True)
    print(f"\n🔍 Running diagnostics for {model_name}...\n")

    # --- R-hat and ESS ---
    summary = az.summary(idata)
    rhat_issues = summary["r_hat"].dropna() > 1.05
    ess_bulk_issues = summary["ess_bulk"].dropna() < 200
    ess_tail_issues = summary["ess_tail"].dropna() < 1000

    print("📏 R-hat summary:")
    print(summary["r_hat"].dropna())
    if rhat_issues.any():
        print(f"⚠️ R-hat > 1.05 for: {list(summary[rhat_issues].index)}")
    else:
        print("✅ All R-hat values < 1.05")

    print("\n📏 ESS (bulk) summary:")
    print(summary["ess_bulk"].dropna())
    if ess_bulk_issues.any():
        print(f"⚠️ ESS < 200 for: {list(summary[ess_bulk_issues].index)}")
    else:
        print("✅ All ESS values > 200")

    print("\n📏 ESS (tail) summary:")
    print(summary["ess_tail"].dropna())
    if ess_tail_issues.any():
        print(f"⚠️ ESS < 1000 for: {list(summary[ess_tail_issues].index)}")
    else:
        print("✅ All ESS values > 1000")

    # --- Divergences ---
    if "diverging" in idata.sample_stats:
        n_divergences = idata.sample_stats["diverging"].sum().values
        print(f"\n🚨 Divergences: {n_divergences}")
        if n_divergences > 0:
            print("⚠️ Consider increasing `target_accept` or reparameterizing.")
        else:
            print("✅ No divergences.")
    else:
        print("ℹ️ Divergence info not available.")

    # --- Tree Depth ---
    if "depth" in idata.sample_stats:
        max_depth = idata.sample_stats["depth"].max().values
        print(f"\n🌲 Max tree depth reached: {max_depth}")
        if max_depth >= 10:
            print("⚠️ Consider increasing `max_treedepth` if sampling is inefficient.")
        else:
            print("✅ Tree depth is within acceptable range.")
    else:
        print("ℹ️ Tree depth info not available.")

    # --- Energy plot (E-BFMI) ---
    energy_plot_path = os.path.join(output_dir, f"{model_name.lower()}_energy_plot.png")
    az.plot_energy(idata)
    plt.title("Energy Transition Plot (E-BFMI)")
    plt.tight_layout()
    plt.savefig(energy_plot_path, dpi=300)
    plt.close()
    print(f"\n📈 Saved energy transition plot to: {energy_plot_path}")

    # --- Optional: Posterior Predictive Check Plot ---
    if hasattr(idata, "posterior_predictive") and "y_pred" in idata.posterior_predictive:
        ppc_plot_path = os.path.join(output_dir, f"{model_name.lower()}_ppc_check.png")
        az.plot_ppc(idata)
        plt.title("Posterior Predictive Check")
        plt.tight_layout()
        plt.savefig(ppc_plot_path, dpi=300)
        plt.close()
        print(f"📈 Saved PPC plot to: {ppc_plot_path}")

    print(f"\n✅ Diagnostic check complete for {model_name}.\n")


### Testing the model using the survey dataset

In [8]:
# Survey Data Generation
%run code_snippets.py

25-26/77-78 27-28/79-80 29-30/81-82 31-32/83-84 33-34/85-86 35-36/87-88 37-38/89-90 39-40/91-92 41-42/93-94 43-44/95-96


In [9]:
d.columns

Index(['Unnamed: 0', 'Idő', 'Befejezés', 'Forrás',
       'Változtatott-e önszántából munkahelyet 2022. január 1. óta?',
       'Rendezze sorrendbe fontosság szerint az alábbi, a munkához kapcsolódó értékeket. A sorba rendezéshez használjon csúszkát, ne a nyilakat!\nMit tartott elsősorban, másodsorban stb. fontosnak, amikor munkahelyet változtatott? x Kevés stresszel járó munka',
       'Rendezze sorrendbe fontosság szerint az alábbi, a munkához kapcsolódó értékeket. A sorba rendezéshez használjon csúszkát, ne a nyilakat!\nMit tartott elsősorban, másodsorban stb. fontosnak, amikor munkahelyet változtatott? x Hosszú szabadság',
       'Rendezze sorrendbe fontosság szerint az alábbi, a munkához kapcsolódó értékeket. A sorba rendezéshez használjon csúszkát, ne a nyilakat!\nMit tartott elsősorban, másodsorban stb. fontosnak, amikor munkahelyet változtatott? x Szakmai fejlődési lehetőség',
       'Rendezze sorrendbe fontosság szerint az alábbi, a munkához kapcsolódó értékeket. A sorba rende

In [10]:
# Rename columns to C999 format
new_columns = {old_col: f'C{i:03d}' for i, old_col in enumerate(d.columns)}
d = d.rename(columns=new_columns)

In [11]:
#Merge the predictor columns
d["I_I"] = merge_columns(d["C027"], d["C079"])
print("\nDataframe with merged \"I_I\" columns (first 5 rows of relevant columns):")
print(d[["C027", "C079", "I_I"]].head())


Dataframe with merged "I_I" columns (first 5 rows of relevant columns):
              C027             C079              I_I
0              NaN       Egyetértek       Egyetértek
1              NaN       Egyetértek       Egyetértek
2  Nem értek egyet              NaN  Nem értek egyet
3              NaN  Nem értek egyet  Nem értek egyet
4              NaN  Nem értek egyet  Nem értek egyet


In [12]:
#Prepare the outcome variable
print("\nUnique values in the outcome columns (C004):")
print(d["C004"].unique())


Unique values in the outcome columns (C004):
['Nem' 'Igen']


In [13]:
d["outcome_binary"] = d["C004"].apply(lambda x: 1 if x == "Igen" else 0)
print("\nFirst few rows with binary outcome:")
print(d[["C004", "outcome_binary"]].head())


First few rows with binary outcome:
   C004  outcome_binary
0   Nem               0
1   Nem               0
2  Igen               1
3   Nem               0
4   Nem               0


In [14]:
#Prepare the predictor variable (I_I)
print("\nUnique values in the merged 'I_I' column:")
print(d["I_I"].unique())


Unique values in the merged 'I_I' column:
['Egyetértek' 'Nem értek egyet' 'Teljesen egyetértek'
 'Egyáltalán nem értek egyet']


In [15]:
#Change values of the likert scale to english
mapping = {'Egyetértek': 'I Agree', 'Egyáltalán nem értek egyet': 'I Completely Disagree', 'Nem értek egyet': 'I Do Not Agree', 'Teljesen egyetértek': 'I totally Agree'}
d["I_I"] = d["I_I"].map(mapping)
print("\nUnique values in the 'I_I' column after mapping:")
print(d["I_I"].unique())


Unique values in the 'I_I' column after mapping:
['I Agree' 'I Do Not Agree' 'I totally Agree' 'I Completely Disagree']


In [16]:
d.describe()

,C005,C006,C007,C008,C009,C010,C011,C012,C013,C014,...,C068,C069,C070,C071,C072,C073,C074,C075,C076,outcome_binary
count,125.000000,125.000000,125.000000,125.00000,125.000000,125.000000,125.000000,125.000000,125.000000,125.000000,...,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,333.000000
mean,14.648000,10.200000,16.688000,12.88800,13.352000,14.024000,16.904000,10.472000,12.976000,11.760000,...,7.817308,7.456731,9.658654,8.533654,5.875000,4.687500,8.879808,8.721154,6.408654,0.375375
std,4.882913,5.840625,3.798166,4.94847,4.924029,3.766285,2.613316,4.363662,3.670862,4.135605,...,4.417279,4.014232,4.655662,4.684110,4.111225,3.955481,5.625166,5.349535,6.019399,0.484948
min,2.000000,1.000000,4.000000,1.00000,1.000000,3.000000,9.000000,1.000000,4.000000,1.000000,...,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,11.000000,5.000000,15.000000,10.00000,11.000000,12.000000,15.000000,7.000000,10.000000,9.000000,...,4.000000,5.000000,6.000000,5.000000,3.000000,2.000000,3.000000,4.000000,1.000000,0.000000
50%,16.000000,10.000000,18.000000,14.00000,14.000000,15.000000,17.000000,11.000000,13.000000,11.000000,...,8.000000,7.000000,9.000000,7.000000,5.000000,4.000000,9.000000,9.000000,4.000000,0.000000
75%,19.000000,15.000000,20.000000,17.00000,17.000000,17.000000,19.000000,13.000000,16.000000,14.000000,...,11.000000,10.000000,13.250000,12.000000,8.000000,7.000000,14.000000,13.000000,11.000000,1.000000
max,20.000000,20.000000,20.000000,20.00000,20.000000,20.000000,20.000000,19.000000,20.000000,20.000000,...,19.000000,19.000000,20.000000,19.000000,17.000000,20.000000,20.000000,20.000000,20.000000,1.000000


In [17]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Columns: 114 entries, C000 to outcome_binary
dtypes: float64(40), int64(1), object(73)
memory usage: 296.7+ KB


In [18]:
d.isna().sum()

C000               0
C001               0
C002               0
C003               0
C004               0
                  ..
C109              24
C110              25
C111               0
I_I                0
outcome_binary     0
Length: 114, dtype: int64

In [19]:
likert_mapping = {
    "I Completely Disagree": 1,
    "I Do Not Agree": 2,
    "I Agree": 3,
    "I totally Agree": 4
}
d["I_I_numeric"] = d["I_I"].map(likert_mapping)

In [20]:
#Prepare data for the Bayesian model
y = d["outcome_binary"].values
X = d["I_I_numeric"].values

In [21]:
d_model = pd.DataFrame({
        "predictor": X,
        "outcome": y
    })
d_model.head()

,predictor,outcome
0,3,0
1,3,0
2,2,1
3,2,0
4,2,0


In [27]:
# Run and save results for B02 for survey data
print("\nResults for B02 Model on survey data:")
idata, *_ = run_and_summarize(d_model, ordinal_predictor_binary_outcome_model, label = "b02", shape = 3)
run_model_diagnostics(idata, model_name="B02", output_dir="results")


Results for B02 Model on survey data:


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>NUTS: [effect, sd_fluctuation, intercept, level_effects_diff]
>BinaryGibbsMetropolis: [y_pred]


Output()

Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 2203 seconds.
There were 843 divergences after tuning. Increase `target_accept` or reparameterize.
Sampling: [y_obs]


Output()

C:\Users\Barakat Ogunjoun\AppData\Local\Temp\ipykernel_26136\1115879529.py:119: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.tight_layout()
C:\Users\Barakat Ogunjoun\AppData\Local\Temp\ipykernel_26136\1115879529.py:120: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig(ppc_plot_path, dpi=300)


✅ Saved all outputs for model B02 in 'results' folder.

🔍 Running diagnostics for B02...

📏 R-hat summary:
effect                   1.0
intercept                1.0
level_effects_diff[0]    1.0
level_effects_diff[1]    1.0
level_effects_diff[2]    1.0
                        ... 
p[328]                   1.0
p[329]                   1.0
p[330]                   1.0
p[331]                   1.0
p[332]                   1.0
Name: r_hat, Length: 676, dtype: float64
✅ All R-hat values < 1.05

📏 ESS (bulk) summary:
effect                   1022.0
intercept                4199.0
level_effects_diff[0]    4585.0
level_effects_diff[1]    9059.0
level_effects_diff[2]    3496.0
                          ...  
p[328]                   9974.0
p[329]                   3771.0
p[330]                   9974.0
p[331]                   9974.0
p[332]                   9974.0
Name: ess_bulk, Length: 676, dtype: float64
✅ All ESS values > 200

📏 ESS (tail) summary:
effect                    1239.0
intercept

## Getting Group Size Ratio PastEx to PresentEx

In [29]:
count = d_model['outcome'].value_counts()
ratio_PastEx_to_Present_Ex = count[1]/count[0]
print(f"Ratio of PastEx to PresentEx: {ratio_PastEx_to_Present_Ex}")

Ratio of PastEx to PresentEx: 0.6009615384615384


In [30]:
from scipy.special import expit  # this is the inverse logit
print(expit(0.201))

0.5500815015304399
